# 2. Computing TF-IDF Vectors with Scikit-Learn

## About dataset source

Scikit-Learn provides us with data from Usenent, which is a well-established online collection of discussion forums. These Usenent forums are called newsgroups. Each individual newsgroup focuses on some topic of discussion. That discussion topic is briefly outlined within the newsgroup name. Users within a newsgroup converse by posting messages. These user posts are generally not limited in length. Some of the posts can get quite large. Both the diversity and the varying lengths of the posts will give us a chance to expand our NLP skills. 

In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(remove=('headers','footers'))

The newsgroups object contains posts from 20 different newsgroups. Each newsgroup’s discussion-theme is outlined in its names. We can view these newsgroup names by printing newsgroups.target_names.

In [3]:
print(newsgroups.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


For example, comp.sys.mac.hardware focuses on mac hardware. Meanwhile, comp.sys.ibm.pc_hardware focuses on pc hardware. Categorically, these 2 newsgroups are exceedingly similar. Their only differentiator is whether the computer hardware belongs to a mac or pc. Sometimes, categorical differences are subtle. Boundaries between text topics are fluid, and are not necessarily etched in stone. We’ll need to keep this in mind, later in the section, when we proceed to cluster the newsgroup posts.

### Get newsgroup group

In [20]:
index = 1
print(newsgroups.data[index])
#get newsgroup
origin = newsgroups.target_names[newsgroups.target[index]]
print(f"\nThe post at index {index} first appeared in the '{origin}' group")

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

The post at index 1 first appeared in the 'comp.sys.mac.hardware' group


### Get newsgroup data size

In [21]:
dataset_size = len(newsgroups.data)
print(f"Our dataset contains {dataset_size} newsgroup posts")

Our dataset contains 11314 newsgroup posts


Our dataset contains over 11,000 posts. **Our goal is to cluster these posts by topic.**
Carrying out text clustering on this scale will require computational efficiency. We’ll need to efficiently compute newsgroup-post similarities by representing our text-data as a matrix. To do so, we’ll need to transform each newsgroup post into TF vector. 

## Vectorizing Documents Using Scikit-Learn

Scikit-Learn provides built-in class for transforming input texts into TF vectors. That class is called CountVectorizer. Initializing CountVectorizer will create a vectorizer object capable of vectorizing our texts

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

All we need to do is run vectorizer.fit_transform(newsgroups.data). The method-call will return the TF matrix corresponding to the vectorized newsgroup posts. As a reminder, a TF matrix stores the counts of words (columns) across all texts (rows).

In [23]:
tf_matrix = vectorizer.fit_transform(newsgroups.data)
print(tf_matrix)

  (0, 108644)	4
  (0, 110106)	1
  (0, 57577)	2
  (0, 24398)	2
  (0, 79534)	1
  (0, 100942)	1
  (0, 37154)	1
  (0, 45141)	1
  (0, 70570)	1
  (0, 78701)	2
  (0, 101084)	4
  (0, 32499)	4
  (0, 92157)	1
  (0, 100827)	6
  (0, 79461)	1
  (0, 39275)	1
  (0, 60326)	2
  (0, 42332)	1
  (0, 96432)	1
  (0, 67137)	1
  (0, 101732)	1
  (0, 27703)	1
  (0, 49871)	2
  (0, 65338)	1
  (0, 14106)	1
  :	:
  (11313, 55901)	1
  (11313, 93448)	1
  (11313, 97535)	1
  (11313, 93393)	1
  (11313, 109366)	1
  (11313, 102215)	1
  (11313, 29148)	1
  (11313, 26901)	1
  (11313, 94401)	1
  (11313, 89686)	1
  (11313, 80827)	1
  (11313, 72219)	1
  (11313, 32984)	1
  (11313, 82912)	1
  (11313, 99934)	1
  (11313, 96505)	1
  (11313, 72102)	1
  (11313, 32981)	1
  (11313, 82692)	1
  (11313, 101854)	1
  (11313, 66399)	1
  (11313, 63405)	1
  (11313, 61366)	1
  (11313, 7462)	1
  (11313, 109600)	1


Our printed tf_matrix does not appear to be a NumPy array. What sort of data structure is it? We can check, by printing type(tf_matrix).

In [24]:
print(type(tf_matrix))

<class 'scipy.sparse.csr.csr_matrix'>


### About csr matrix 

 stands for Compressed Sparse Row, which is a storage format for compressing matrices that are composed mostly of zeros. **These mostly empty matrices are referred to as sparse matrices. They can be made smaller by storing only the non-zero elements. This compression leads to more efficient memory usage, and also faster computation. Large-scale text-based matrices are usually very sparse, since a single document normally contains just a small percentage of the total vocabulary**. Thus, Scikit-Learn automatically converts the vectorized text to the CSR format. The conversion is carried out using a csr_matrix class that’s imported from SciPy.

In order to minimize confusion between csr_matrix and numpy we convert matrix to numpy array 

In [26]:
tf_np_matrix = tf_matrix.toarray()
print(tf_np_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


* Rows - representation of the post
* Cols - represent individual words
* Vocab_size = cols 

In [28]:
assert tf_np_matrix.shape == tf_matrix.shape
num_posts, vocabulary_size = tf_np_matrix.shape
print(f"Our collection of {num_posts} newsgroup posts contain a total of "
      f"{vocabulary_size} unique words")

Our collection of 11314 newsgroup posts contain a total of 114751 unique words


Our data contains 114,000 unique words. However, most posts will hold only a few dozen of these words. **We can measure the unique word-count of a post at index i by counting the number of non-zero elements in row tf_np_matrix[i].** The easiest way to count these non-zero elements is with NumPy. The library allows us to obtain all non-zero indices of the vector at tf_np_matrix[i]. We simply need to input the vector into the np.flatnonzero function. Below, we’ll count and output the non-zero indices of the car post in newsgroups.data[0].

In [40]:
import numpy as np
row_n = 0
tf_vector = tf_np_matrix[row_n]
non_zero_indices = np.flatnonzero(tf_vector)
num_unique_words = non_zero_indices.size
print(f"The newsgroup in row {row_n} contains {num_unique_words} unique words.")
print("The actual word-counts map to the following column indices:\n")
print(non_zero_indices)


The newsgroup in row 0 contains 64 unique words.
The actual word-counts map to the following column indices:

[ 14106  15549  22088  23323  24398  27703  29357  30093  30629  32194
  32305  32499  37154  39275  42332  42333  43643  45089  45141  49871
  49881  50165  54442  55453  57577  58321  58842  60116  60326  64083
  65338  67137  67140  68931  69080  70570  72915  75280  78264  78701
  79055  79461  79534  82759  84398  87690  89161  92157  93304  95225
  96145  96432 100406 100827 100942 101084 101732 108644 109086 109254
 109294 110106 112936 113262]


#### Okay cool... but what are these words ? 

In order to find out, we’ll need a mapping between TF vector indices and word-values. That mapping can be generated by calling vectorizer.get_feature_names(). The method-call will return a list of words, which we’ll call words. Each index i will correspond to the ith word within the list. Thus, running [words[i] for i in non_zero_indices] will return all unique words within our post.

In [41]:
words = vectorizer.get_feature_names()
unique_words = [words[i] for i in non_zero_indices]
print(unique_words)

['60s', '70s', 'addition', 'all', 'anyone', 'be', 'body', 'bricklin', 'bumper', 'called', 'can', 'car', 'could', 'day', 'door', 'doors', 'early', 'engine', 'enlighten', 'from', 'front', 'funky', 'have', 'history', 'if', 'in', 'info', 'is', 'it', 'know', 'late', 'looked', 'looking', 'made', 'mail', 'me', 'model', 'name', 'of', 'on', 'or', 'other', 'out', 'please', 'production', 'really', 'rest', 'saw', 'separate', 'small', 'specs', 'sports', 'tellme', 'the', 'there', 'this', 'to', 'was', 'were', 'whatever', 'where', 'wondering', 'years', 'you']


We’ve printed all the words in newsgroups.data[0]. **Of course, not all these words have equal mention-counts. Some words occur more frequently than others. Perhaps these frequent words are more relevant to the topic of cars.** Lets print the 10 most frequent words within the post, along with their associated counts. We’ll represent this output as a Pandas table, for visualization purposes.

## Extracting Non-Zero Elements of 1D NumPy Arrays

* non_zero_indices = np.flatnonzero(np_vector): Returns the non-zero indices in a 1D NumPy array.
* non_zero_vector = np_vector[non_zero_indices]: Selects the non-zero elements of a 1D NumPy array (assuming non_zero_indices corresponds to non-zero indices of that array).

In [44]:
import pandas as pd

In [47]:
data = {
    "Data":unique_words,
    "Counts":tf_vector[non_zero_indices]
}

In [56]:
df = pd.DataFrame(data).sort_values("Counts", ascending=False)
df.head(10)

,Data,Counts
53,the,6
55,this,4
57,was,4
11,car,4
24,if,2
27,is,2
28,it,2
19,from,2
39,on,2
4,anyone,2


### Whats a noise...

Three first words, have nothing to do with cars. These words, the, this, and was, are among the most common words in the English language. They don’t provide a differentiating signal between the car-post and a differently-themed post.Instead, the common words are a source of noise. They increase the likelihood that 2 unrelated documents will cluster together.

NLP practitioners refer to such noisy words as **stop words**,because these words are blocked from appearing in the vectorized results. Stop words are generally deleted from the text prior to vectorization. T

Below, we’ll re-initialize a stop-word aware vectorizer. Afterwards, we’ll rerun fit_transfrom in order to re-compute the TF matrix. The number of word-columns in that matrix will be less than our previously computed vocabulary size of 114,751. Also, we’ll regenerate our words list. Common stop words such as the, this, of and it will be missing from that list.

In [57]:
vectorizer = CountVectorizer(stop_words='english')
tf_matrix = vectorizer.fit_transform(newsgroups.data)
assert tf_matrix.shape[1] < 114751

words = vectorizer.get_feature_names()
for common_word in ['the', 'this', 'was', 'if', 'it', 'on']:
    assert common_word not in words

All stop words have been deleted from the recomputed tf_matrix. Now, we can re-generate the 10 most frequent words in newsgroups.data[0]. Please note that in the process, we’ll recompute tf_np_matrix, tf_vector, unique_words, non_zero_indices, and df.

### Top words after stop-word deletion

In [61]:
text_index = 0
tf_np_matrix = tf_matrix.toarray()
tf_vector = tf_np_matrix[text_index]
non_zero_indices = np.flatnonzero(tf_vector)
unique_words = [words[index] for index in non_zero_indices]
data = {'Word': unique_words,
        'Count': tf_vector[non_zero_indices]}

df = pd.DataFrame(data).sort_values('Count', ascending=False)
print(f"After stop-word deletion, {df.shape[0]} unique words remain.")
print("The 10 most frequent words are:\n")
print(df[:10].to_string(index=False))

After stop-word deletion, 34 unique words remain.
The 10 most frequent words are:

       Word  Count
        car      4
        60s      1
        saw      1
    looking      1
       mail      1
      model      1
 production      1
     really      1
       rest      1
   separate      1


However, it’s worth noting that not all words are equal in their relevancy. Some words are more relevant to a car-discussion than others. For instance, the word model refers to a car-model (though of course it could also refer to a supermodel or a machine learning model). Meanwhile, the word really is more general. It doesn’t refer to anything car-related. The word is so irrelevant and common, that it could almost be a stop word. In fact, some NLP practitioners keep really on their stop-word list, while others don’t. Unfortunately, there is no shared consensus on which words are always useless, and which words aren’t. However, all practitioners agree that a word becomes less useful if its mentioned in too many texts. Thus, really is less relevant than model, because the former is mentioned more posts. Therefore, when ranking words by relevance, we should leverage both post-frequency and count. If two words share an equal count, then we should rank them by post-frequency instead.

Lets rerank our 34 words based on on both post-frequency and count. Afterwards, we’ll explore how these rankings can be used to improve text-vectorization.

< Screen z najczęściej wykorzystywanymi metodami Count Vectorizera>

## Ranking Words by Both Post-Frequency and Count
